In [17]:
# Import packages to use
import twint
import numpy as np
import pickle
import pandas as pd
import time
import math
from collections import Counter
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
%matplotlib inline

# Patches asyncio to allow the running of multiple event loops in Jupyter Notebooks.
# Fixes: "RuntimeError: This event loop is already running"
import nest_asyncio

nest_asyncio.apply()
print(pd.__version__)

1.0.1


In [18]:
# Finds the distance between two lat/long coordinates
def haversine_distance(lat1, lon1, lat2, lon2):
    r = 6371
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
    res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
    return np.round(res, 2)

## Collect Users

In [19]:
def getSantaColomaTweets(start_date, end_date):
    df = pd.DataFrame()

    # Search Tweets with the various buildings and regions near Santa Coloma - Part 1
    c = twint.Config()
    c.Search = '\"Santa Coloma de Gramenet\" OR \"Santa Coloma\" OR \"Rambla San Sebastian\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    
    # Search Tweets with the various buildings and regions near Santa Coloma - Part 2
    c = twint.Config()
    c.Search = '\"Parque Fluvial del Besos\" OR \"Parc Fluvial del Besos\" OR \"Fluvial del Besos\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with the various buildings and regions near Santa Coloma - Part 3
    c = twint.Config()
    c.Search = '\"Parque Molinet\" OR \"Parc Molinet\" OR \"Molinet\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with the various buildings and regions near Santa Coloma - Part 4
    c = twint.Config()
    c.Search = '\"Plaza del Rellotge\" OR \"Plaça del Rellotge\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with the various buildings and regions near Santa Coloma - Part 5
    c = twint.Config()
    c.Search = '\"Parque Can Zam\" OR \"Parc Can Zam\" OR \"Can Zam\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with the various buildings and regions near Santa Coloma - Part 6
    c = twint.Config()
    c.Search = '\"Instituto Can Peixauet\" OR \"Institut Can Peixauet\" OR \"Can Peixauet\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with the various buildings and regions near Santa Coloma - Part 7
    c = twint.Config()
    c.Search = '\"Parque Gran Sol\" OR \"Parc Gran Sol\" OR \"Gran Sol\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with the various buildings and regions near Santa Coloma - Part 8
    c = twint.Config()
    c.Search = '\"Escuela Tanit\" OR \"Escola Tanit\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with the various buildings and regions near Santa Coloma - Part 9
    c = twint.Config()
    c.Search = '\"Instituto Terra Roja\" OR \"Institut Terra Roja\" OR \"Terra Roja\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with the various buildings and regions near Santa Coloma - Part 10
    c = twint.Config()
    c.Search = '\"Instituto Gassol\" OR \"Institut Gassol\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with the various buildings and regions near Santa Coloma - Part 11
    c = twint.Config()
    c.Search = '\"CAP Santa Rosa\" OR \"Cinto Verdaguer\" OR \"Nus de la Trinitat\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with the various buildings and regions near Santa Coloma - Part 12
    c = twint.Config()
    c.Search = '\"Mercado del Fondo\" OR \"Mercat del Fondo\" OR \"del Fondo\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with location near 'Santa Coloma de Gramenet': North Side
    c = twint.Config()
    c.Geo = "41.46287400801948, 2.2028934732857177, 1km"
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with location near 'Santa Coloma de Gramenet': South Side
    c = twint.Config()
    c.Geo = "41.45039468429977, 2.212764002746006, 0.75km"
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    return df

In [20]:
data = getSantaColomaTweets("2019-07-01", "2019-08-02") # effectively searches for dates 07/01 - 07/31

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping w

In [ ]:
# collect users posting in native language of area
native_users = set(data.loc[data['language'].isin(['es', 'ca'])]['username'])
pickle.dump(native_users, open( "native_users.p", "wb" ) )
print("Num of native language users:", len(native_users))

# eliminate users posting in native language of area
users_with_nonnative_lang_tweets = set(data.loc[~data['language'].isin(['es', 'ca', 'und'])]['username'])
foreign_users = users_with_nonnative_lang_tweets - native_users
pickle.dump(foreign_users, open( "foreign_users.p", "wb" ) )
print("Num of nonnative language users:", len(foreign_users))

In [ ]:
# read in pickle file
native_users = pickle.load( open( "native_users.p", "rb" ) )
foreign_users = pickle.load( open( "foreign_users.p", "rb") )

# User Analysis

In [56]:
patterns = {
    "Santa Coloma de Gramanet" : [x.lower() for x in ["Santa Coloma de Gramenet", "Santa Coloma"]],
    "Fluvial del Besos" : [x.lower() for x in ["Parque Fluvial del Besos", "Parc Fluvial del Besos", "Fluvial del Besos", "rio.*besos|besos.*rio"]],
    "Parque Molinet" : [x.lower() for x in ["Parque Molinet", "Parc Molinet", "Molinet"]],
    "Plaza del Rellotge" : [x.lower() for x in ["Plaza del Rellotge", "Plaça del Rellotge"]],
    "Rambla San Sebastian" : [x.lower() for x in ["Rambla San Sebastian"]],
    "Parque Can Zam" : [x.lower() for x in ["Parque Can Zam", "Parc Can Zam", "Can Zam"]],
    "Instituto Can Peixauet" : [x.lower() for x in ["Instituto Can Peixauet", "Institut Can Peixauet", "Can Peixauet"]],
    "Parque Gran Sol" : [x.lower() for x in ["Parque Gran Sol", "Parc Gran Sol", "Gran Sol"]],
    "Escuela Tanit" : [x.lower() for x in ["Escuela Tanit", "Escola Tanit"]],
    "Instituto Terra Roja" : [x.lower() for x in ["Instituto Terra Roja", "Institut Terra Roja", "Terra Roja"]],
    "Instituto Gassol" : [x.lower() for x in ["Instituto Gassol", "Institut Gassol"]],
    "CAP Santa Rosa" : [x.lower() for x in ["CAP Santa Rosa"]],
    "Cinto Verdaguer" : [x.lower() for x in ["Cinto Verdaguer"]],
    "Mercado del Fondo" : [x.lower() for x in ["Mercado del Fondo", "Mercat del Fondo", "del Fondo"]],
    "Nus de la Trinitat" : [x.lower() for x in ["Nus de la Trinitat"]],
    "Macanet str" : [x.lower() for x in ["Maçanet str"]],
    "Iglesia Evangelica" : [x.lower() for x in ["Iglesia Evangelica", "Iglesia Esglesia"]]
}

## Foreign users

In [57]:
foreign_tweets_by_location = {
    "Santa Coloma de Gramanet" : pd.DataFrame(),
    "Fluvial del Besos" : pd.DataFrame(),
    "Parque Molinet" : pd.DataFrame(),
    "Plaza del Rellotge" : pd.DataFrame(),
    "Rambla San Sebastian" : pd.DataFrame(),
    "Parque Can Zam" : pd.DataFrame(),
    "Instituto Can Peixauet" : pd.DataFrame(),
    "Parque Gran Sol" : pd.DataFrame(),
    "Escuela Tanit" : pd.DataFrame(),
    "Instituto Terra Roja" : pd.DataFrame(),
    "Instituto Gassol" : pd.DataFrame(),
    "CAP Santa Rosa" : pd.DataFrame(),
    "Cinto Verdaguer" : pd.DataFrame(),
    "Mercado del Fondo" : pd.DataFrame(),
    "Nus de la Trinitat" : pd.DataFrame(),
    "Macanet str" : pd.DataFrame(),
    "Iglesia Evangelica" : pd.DataFrame()
}

In [ ]:
i = 1
start = time.time()

# iterates through yearlong Tweets for the above selected users and filters users who have been associated with 
# Santa Coloma within 2 weeks 
for user in foreign_users:
    c = twint.Config()
    c.Username = user
    c.Since = "2019-01-01"
    c.Until = "2019-12-31"
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)

    df2 = twint.storage.panda.Tweets_df
    if (len(df2) > 0): 
        df2['tweet'] = df2['tweet'].str.lower()

        for k, location_df in foreign_tweets_by_location.items():
            pattern = '|'.join(patterns[k])

            tweets = df2[df2['tweet'].str.contains(pattern)]
            
            location_df = location_df.append(tweets)
            location_df = location_df.drop_duplicates(subset=['id'], ignore_index=True)
            foreign_tweets_by_location[k] = location_df
            
#             df2_tweets = df2_tweets.append(tweets)
#             df2_tweets = df2_tweets.drop_duplicates(subset=['id'], ignore_index=True)


#         if len(df2_tweets) > 0:
#             max_date = datetime.strptime(max(df2_tweets['date']), "%Y-%m-%d %H:%M:%S")
#             min_date = datetime.strptime(min(df2_tweets['date']), "%Y-%m-%d %H:%M:%S")

#             if max_date - min_date < timedelta(days=14):
#                 foreign_less_than_two_weeks.append(user)


#         df2_places = df2.loc[(df2['place'] != ''), ['username','place','date']]
#         if len(df2_places) > 0:
#             df2_places['coordinates'] = [x['coordinates'] for x in df2_places['place']]
#             # get distances to the two central points of Santa Coloma
#             df2_places['dist1'] = [haversine_distance(*x, 41.45039468429977, 2.212764002746006) for x in df2_places['coordinates']]
#             df2_places['dist2'] = [haversine_distance(*x, 41.46287400801948, 2.2028934732857177) for x in df2_places['coordinates']]
#             df2_places = df2_places[(df2_places['dist1'] < 0.75) | (df2_places['dist2'] < 1.0)]

#             if len(df2_places) > 0:
#                 max_date = datetime.strptime(max(df2_places['date']), "%Y-%m-%d %H:%M:%S")
#                 min_date = datetime.strptime(min(df2_places['date']), "%Y-%m-%d %H:%M:%S")

#                 if max_date - min_date < timedelta(days=14):
#                     foreign_places.append(user)

    print(str(math.floor(i * 100 / len(foreign_users))) + "% done. Time to completion: "
          + str(round(((time.time() - start) / i) * (len(foreign_users) - i), 0)) + " seconds.")
    i += 1
print("Total time of completion: " + str(time.time() - start) + " seconds.")

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 3766.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 18712.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 14547.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 12099.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 11430.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 9702.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 8469.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 7667

In [18]:
#original freq data
freq

{'Santa Coloma de Gramanet': 991,
 'Fluvial del Besos': 0,
 'Parque Molinet': 118,
 'Plaza del Rellotge': 5,
 'Rambla San Sebastian': 0,
 'Parque Can Zam': 76,
 'Instituto Can Peixauet': 7,
 'Parque Gran Sol': 329,
 'Escuela Tanit': 0,
 'Instituto Terra Roja': 2,
 'Instituto Gassol': 0,
 'CAP Santa Rosa': 0,
 'Cinto Verdaguer': 1,
 'Mercado del Fondo': 2061,
 'Nus de la Trinitat': 1}

In [25]:
foreign_freq

{'Santa Coloma de Gramanet': 597,
 'Fluvial del Besos': 16,
 'Parque Molinet': 107,
 'Plaza del Rellotge': 2,
 'Rambla San Sebastian': 0,
 'Parque Can Zam': 32,
 'Instituto Can Peixauet': 3,
 'Parque Gran Sol': 44,
 'Escuela Tanit': 0,
 'Instituto Terra Roja': 1,
 'Instituto Gassol': 0,
 'CAP Santa Rosa': 0,
 'Cinto Verdaguer': 1,
 'Mercado del Fondo': 1777,
 'Nus de la Trinitat': 0,
 'Macanet str': 0,
 'Iglesia Evangelica': 0}

In [28]:
foreign_user_count

{'Santa Coloma de Gramanet': 135,
 'Fluvial del Besos': 6,
 'Parque Molinet': 41,
 'Plaza del Rellotge': 1,
 'Rambla San Sebastian': 0,
 'Parque Can Zam': 7,
 'Instituto Can Peixauet': 2,
 'Parque Gran Sol': 37,
 'Escuela Tanit': 0,
 'Instituto Terra Roja': 1,
 'Instituto Gassol': 0,
 'CAP Santa Rosa': 0,
 'Cinto Verdaguer': 1,
 'Mercado del Fondo': 593,
 'Nus de la Trinitat': 0,
 'Macanet str': 0,
 'Iglesia Evangelica': 0}

In [ ]:
pickle.dump(foreign_tweets_by_location, open( "foreign_freq.p", "wb" ) )

In [ ]:
foreign_tweets_by_location = pickle.load( open( "foreign_freq.p", "rb" ) )

## Native Users

In [ ]:
native_tweets_by_location = {
    "Santa Coloma de Gramanet" : pd.DataFrame(),
    "Fluvial del Besos" : pd.DataFrame(),
    "Parque Molinet" : pd.DataFrame(),
    "Plaza del Rellotge" : pd.DataFrame(),
    "Rambla San Sebastian" : pd.DataFrame(),
    "Parque Can Zam" : pd.DataFrame(),
    "Instituto Can Peixauet" : pd.DataFrame(),
    "Parque Gran Sol" : pd.DataFrame(),
    "Escuela Tanit" : pd.DataFrame(),
    "Instituto Terra Roja" : pd.DataFrame(),
    "Instituto Gassol" : pd.DataFrame(),
    "CAP Santa Rosa" : pd.DataFrame(),
    "Cinto Verdaguer" : pd.DataFrame(),
    "Mercado del Fondo" : pd.DataFrame(),
    "Nus de la Trinitat" : pd.DataFrame(),
    "Macanet str" : pd.DataFrame(),
    "Iglesia Evangelica" : pd.DataFrame()
}
native_users_list = list(native_users)

In [ ]:
def getNativeUserFrequencies(user_list, tweets_by_location):
    i = 1
    start = time.time()

    # iterates through yearlong Tweets for the above selected users and filters users who have been associated with 
    # Santa Coloma within 2 weeks 
    for user in user_list:
        c = twint.Config()
        c.Username = user
        c.Since = "2019-01-01"
        c.Until = "2019-12-31"
        c.Pandas = True
        c.Hide_output = True
        twint.run.Search(c)

        df2 = twint.storage.panda.Tweets_df
        if (len(df2) > 0): 
            df2['tweet'] = df2['tweet'].str.lower()

            for k, location_df in tweets_by_location.items():
                pattern = '|'.join(patterns[k])

                tweets = df2[df2['tweet'].str.contains(pattern)]

#                 df2_tweets = df2_tweets.append(tweets)
#                 df2_tweets = df2_tweets.drop_duplicates(subset=['id'], ignore_index=True)
                location_df = location_df.append(tweets)
                location_df = location_df.drop_duplicates(subset=['id'], ignore_index=True)
                tweets_by_location[k] = location_df


#             if len(df2_tweets) > 0:
#                 max_date = datetime.strptime(max(df2_tweets['date']), "%Y-%m-%d %H:%M:%S")
#                 min_date = datetime.strptime(min(df2_tweets['date']), "%Y-%m-%d %H:%M:%S")

#                 if max_date - min_date < timedelta(days=14):
#                     less_than_two_weeks.append(user)


#             df2_places = df2.loc[(df2['place'] != ''), ['username','place','date']]
#             if len(df2_places) > 0:
#                 df2_places['coordinates'] = [x['coordinates'] for x in df2_places['place']]
#                 # get distances to the two central points of Santa Coloma
#                 df2_places['dist1'] = [haversine_distance(*x, 41.45039468429977, 2.212764002746006) for x in df2_places['coordinates']]
#                 df2_places['dist2'] = [haversine_distance(*x, 41.46287400801948, 2.2028934732857177) for x in df2_places['coordinates']]
#                 df2_places = df2_places[(df2_places['dist1'] < 0.75) | (df2_places['dist2'] < 1.0)]

#                 if len(df2_places) > 0:
#                     max_date = datetime.strptime(max(df2_places['date']), "%Y-%m-%d %H:%M:%S")
#                     min_date = datetime.strptime(min(df2_places['date']), "%Y-%m-%d %H:%M:%S")

#                     if max_date - min_date < timedelta(days=14):
#                         places.append(user)

        print(str(math.floor(i * 100 / len(user_list))) + "% done. Time to completion: "
              + str(round(((time.time() - start) / i) * (len(user_list) - i), 0)) + " seconds.")
        i += 1
    print("Total time of completion: " + str(time.time() - start) + " seconds.")
    
    return tweets_by_location

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[0:1000], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[1000:2000], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[2000:3000], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[3000:4000], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[4000:5000], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[5000:6000], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[6000:7000], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[7000:8000], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[8000:9000], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[9000:10000], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[10000:11000], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[11000:12000], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[12000:13000], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[13000:14000], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[14000:], native_tweets_by_location)

# Location Based

In [ ]:
c = twint.Config()
c.Search = '\"Santa Coloma de Gramenet\" OR \"Santa Coloma\" OR \"Rambla San Sebastian\"'
c.Since = "2019-01-01"
c.Until = "2019-01-03"
c.Pandas = True
c.Hide_output = True
twint.run.Search(c)

df = twint.storage.panda.Tweets_df